In [5]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

import pandas as pd

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [6]:
df = pd.read_excel('Exercise_Files/CCPP/Folds5x2_pp.xlsx', sheet_name='Sheet1')
pp_df = spark.createDataFrame(df)

In [8]:
pp_df.take(1)

[Row(AT=14.96, V=41.76, AP=1024.07, RH=73.17, PE=463.26)]

In [9]:
vectorAssembler = VectorAssembler(inputCols=["AT", "V", "AP", "RH"], outputCol="features")

In [11]:
vpp_df = vectorAssembler.transform(pp_df)

In [12]:
vpp_df.take(1)

[Row(AT=14.96, V=41.76, AP=1024.07, RH=73.17, PE=463.26, features=DenseVector([14.96, 41.76, 1024.07, 73.17]))]

In [14]:
splits = vpp_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [18]:
train_df.count()

6686

In [16]:
test_df.count()

2882

In [17]:
vpp_df.count()

9568

In [19]:
dt = DecisionTreeRegressor(featuresCol="features", labelCol="PE")

In [22]:
dt_model = dt.fit(train_df)

In [23]:
dt_predictions = dt_model.transform(test_df)

In [24]:
dt_evaluator = RegressionEvaluator(labelCol="PE", predictionCol="prediction", metricName="rmse")

In [26]:
rmse = dt_evaluator.evaluate(dt_predictions)  # root-mean-squared-error

In [27]:
rmse

4.496111531306097